In [1]:
from tagger.core.mongo.models.order import Order
from tagger.models import ExtendedOrder, ExtendedOrderItem


import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [2]:
valid_orders = Order.objects(orderstatus__nin=[
    "CREATED",
    "RECREATED",
    "PAYMENT_PENDING"
]).all()

In [3]:
for o in valid_orders:
    ExtendedOrder.make(o, force_update=True)

order items not extended correctly for eo 6RNQL.. purging and remaking
6RNQL - SANDRO___OSAN배색가디건___1
order items not extended correctly for eo JAW9N.. purging and remaking
JAW9N - BEANPOLEL___울캐시미어케이프핸드메이드리버시블코트___FREE
order items not extended correctly for eo 4CR8M.. purging and remaking
4CR8M - BEANPOLEL___리버시블체크후드케이프코트___FREE


In [4]:
# 6JUTV

X9YJM
61303c68a93cdd965c8cf222
X9YJM


In [ ]:
for o in valid_orders:
    if ExtendedOrder.objects.filter(order_id=o.id).count() > 0:
        eo = ExtendedOrder.objects.get(order_id=o.id)
        if eo.code != code and code is not None:
            print(f"Changing extended order {o.id} code from {eo.code} to {code}")
            eo.code = code
            eo.save()
        elif eo.code is None:
            new_code = ExtendedOrder.generate_usable_code()
            print(f"Changing extended order {o.id} code from {eo.code} to {new_code}")
            eo.code = new_code
            eo.save()
    else:
        e = ExtendedOrder.make_from(o, code=code)
        print(f"Order {o.id} has been extended with code {e.code}! (Given code: {code})")